<a href="https://colab.research.google.com/github/lcipolina/Ray_tutorials/blob/main/MARL-custom_action.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "ray[rllib,tune]" gym

In [ ]:
import os
import numpy as np
import ray
from ray.rllib.env import MultiAgentEnv
from gym.spaces import MultiDiscrete
from ray import air, tune
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.utils.spaces.simplex import Simplex                                 # for the Dirichlet action space //# Simplex -> Dirichlet.
from ray.rllib.models.torch.torch_action_dist import TorchDirichlet as Dirichlet   # the dirichlet action-definition wrapper
from ray.rllib.models import ModelCatalog                                          #  we need to register the new action space


In [2]:
# Define custom env
class SimpleEnv(MultiAgentEnv):

    def __init__(self, config=None):
        super().__init__()
        self.num_agents              = 2
        self.steps                   = 0
        self.agent_lst               = list(range(self.num_agents))
        self._agent_ids              = set(self.agent_lst)
        self.dones_dict              = {agent: False for agent in self.agent_lst}
        self.dones_dict['__all__']   = False
        self.observation_space       = Simplex((self.num_agents,)) 
        self.action_space            = self.observation_space

        
    def reset(self):
        return {0: self.observation_space.sample()}  # {agent:obs}


    def step(self, action_dict):
        return {1:self.observation_space.sample()}, {1:0}, self.dones_dict, {} #{agent:obs}, {agent:rews}, dones, info

In [3]:
# Register custom environment and action_space
env_name = 'SimpleEnv'
tune.register_env(env_name, lambda env_ctx: SimpleEnv()) #the register_env needs a callable/iterable
ModelCatalog.register_custom_action_dist("Dirichlet", Dirichlet)  #uses the imported one. Name string will be passed to Config dict

In [4]:
#RLLIB config experiment 

NUM_CPUS = os.cpu_count()
train_steps = 1


config = PPOConfig()\
    .framework("torch")\
    .rollouts(num_rollout_workers=1, )\
    .resources(num_gpus=0,num_cpus_per_worker=1)\
    .evaluation(evaluation_interval=2,evaluation_duration = 2, evaluation_duration_unit='episodes',
                evaluation_config= {"explore": False})\
    .environment(env = 'SimpleEnv', env_config={
                                     "num_workers": NUM_CPUS - 1,
                                     "disable_env_checking":True} #env_config: arguments passed to the Env + num_workers = # number of parallel workers

                 )\
    .training(model = {"custom_action_dist": "Dirichlet"})

In [5]:
# RLLIB Train

if ray.is_initialized(): ray.shutdown()
ray.init(local_mode=True,include_dashboard=False, ignore_reinit_error=True,) #If dashboard True, prints the dashboard running on a local port

train_steps = 1
stop_timesteps = 3
stop_iters = 3
stop_dict = {
        "timesteps_total": stop_timesteps,
        "training_iteration": stop_iters,
    }

experiment_name = 'test'
tuner = tune.Tuner("PPO", param_space=config,
                              run_config=air.RunConfig(
                                        name =  experiment_name,
                                        stop = stop_dict,
                                )
                                  )
results = tuner.fit()
ray.shutdown()

/usr/local/lib/python3.9/dist-packages/ray/_private/client_mode_hook.py:105: DeprecationWarning: DeprecationWarning: local mode is an experimental feature that is no longer maintained and will be removed in the future.For debugging consider using Ray debugger. 
  return func(*args, **kwargs)
2023-04-10 14:45:43,221	INFO worker.py:1553 -- Started a local Ray instance.


:job_id:01000000
2023-04-10 14:45:46,884	WARNING worker.py:846 -- `ray.get_gpu_ids()` will always return the empty list when called from the driver. This is because Ray does not manage GPU allocations to the driver process.
:task_name:bundle_reservation_check_func
:actor_name:PPO
2023-04-10 14:45:47,020	WARNING algorithm_config.py:596 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
2023-04-10 14:45:47,031	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


:job_id:01000000
:task_name:bundle_reservation_check_func
:actor_name:PPO


:actor_name:RolloutWorker
2023-04-10 14:45:47,186	WARNING env.py:296 -- Your MultiAgentEnv <SimpleEnv instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.


:actor_name:RolloutWorker


/usr/local/lib/python3.9/dist-packages/ray/rllib/models/torch/torch_action_dist.py:625: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  self.last_sample = nn.functional.softmax(self.dist.concentration)
2023-04-10 14:45:47,452	ERROR actor.py:968 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=544, ip=172.28.0.12, repr=<ray.rllib.evaluation.rollout_worker._modify_class.<locals>.Class object at 0x7f22b11acc70>)
  File "/usr/local/lib/python3.9/dist-packages/ray/rllib/evaluation/rollout_worker.py", line 737, in __init__
    self._build_policy_map(policy_dict=self.policy_dict)
  File "/usr/local/lib/python3.9/dist-packages/ray/rllib/evaluation/rollout_worker.py", line 1984, in _build_policy_map
    new_policy = create_policy_for_framework(
  File "/usr/local/lib/python3.9/dist-packages/ray/rllib/utils/policy.py", line 139, in cre

RayTaskError(AttributeError): ignored